In [1]:
# pip install dspy-ai
# https://medium.com/@frederick.ros/dspy-essentials-leveraging-assertions-for-optimized-data-handling-b9e8769cd087

In [1]:
import dspy
import random

In [2]:
my_name = dspy.Example(transcript="This is a transcript from a conversation",
                       action_items="and extracted action items").with_inputs("transcript")

In [3]:
llm = dspy.OllamaLocal(model='gemma2', temperature=0.0)

In [4]:
dspy.settings.configure(lm=llm)

In [5]:
def varying_temp():
    temperature = 0.5 + 0.0001 * random.uniform(-1, 1)
    return {"config": {"temperature": temperature}}

In [6]:
class TranscriptExample(dspy.Signature):
    """Generate a short conversation transcript a discussion and extracted action_items"""
    transcript = dspy.OutputField(desc="A short transcript of a discussion (can be a monolog, like something you recording on your phone to not forget)")
    action_items = dspy.OutputField(desc="A comma-separated list of action items present in the transcript")

class GenerateExample(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_example = dspy.ChainOfThought(TranscriptExample)

    def forward(self):
        return self.generate_example(**varying_temp())


example_generator = GenerateExample()
example = example_generator()
example2 = example_generator()
example3 = example_generator()

print(example)
print(example2)
print(example3)

Prediction(
    rationale="Reasoning: Let's think step by step in order to  generate a realistic conversation transcript about planning a weekend trip. We need to consider things like destination, activities, budget, and who is going.",
    transcript='"Okay, so we\'re finally taking that weekend trip! I was thinking maybe somewhere with mountains? Hiking would be amazing. What do you guys think?  We could look at places within driving distance, save on gas and flights.  Also, how much are we looking to spend? We need to figure out accommodation and food. Oh, and who\'s coming? Just us or inviting some friends?"',
    action_items='- Research mountain destinations within driving distance\n- Determine budget for trip\n-'
)
Prediction(
    rationale='## Conversation Transcript Example \n\n**Reasoning:** Let\'s think step by step in order to  brainstorm ideas for our team\'s upcoming holiday party and figure out who can take on which tasks. We need to consider the theme, location, food, a

In [7]:
import re

def check_format(action_items):
  """Check that the action items are a list of comma-separated action items"""
  # Simple check first: if we split according the , we should have more than 2 items
  if len(action_items.split(",")) == 1:
    return False
  
  # Now let's check it's not formated as 1.action item 1, blablah\n2. Action item 2, ..
  match = re.search(r'(\d)\.\s.+?(\\n|$)', action_items, re.MULTILINE)
  if match:
    return False
    
  return True

class GenerateExampleWithAssert(dspy.Module):
  def __init__(self):
    super().__init__()
    self.generate_example = dspy.ChainOfThought(TranscriptExample)
  
  def forward(self):
    ex = self.generate_example(**varying_temp())
    dspy.Assert(check_format(ex.action_items), "Action Items should be a comma-separated list")

    return ex

In [8]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

dspy.configure(trace=[])

generate_with_assert = assert_transform_module(GenerateExampleWithAssert(), backtrack_handler)

example = generate_with_assert()
example1 = generate_with_assert()
example2 = generate_with_assert()

print(example)
print(example1)
print(example2)


Prediction(
    rationale='##  Conversation Transcript Example \n\n**Reasoning:** Let\'s think step by step in order to identify tasks that need to be completed for the upcoming team presentation. We should brainstorm ideas, assign responsibilities, and set deadlines. \n\n**Transcript:** "Okay, so we have the team presentation next week. I think we should start brainstorming some ideas today. What are everyone\'s thoughts on the topic?  Maybe Sarah, you could lead on finding some data to support our points? And John, since you\'re great with visuals, could you look into creating some slides? As for me, I\'ll focus on outlining the main talking points and scheduling a time for us to rehearse. What do you guys think?"',
    transcript='',
    action_items='Reasoning: Let\'s think step by step in order to  plan our weekend trip. We need to decide on a destination, book accommodations, and make a list of activities. \n\nTranscript: "Hey! So I was thinking about our weekend getaway. Where s

In [9]:
class ActionItemCompliance(dspy.Signature):
    """Check that all action items are included in the text"""
    text = dspy.InputField()
    action_items = dspy.InputField(desc="A comma-separated list of action items")
    comply : bool = dspy.OutputField(desc="True or False")

check_inclusion = dspy.TypedChainOfThought(ActionItemCompliance)

def are_action_items_included(transcript, action_items):
    comp = check_inclusion(text=transcript, action_items=action_items)
    return comp.comply

print(are_action_items_included(example.transcript, example.action_items))

True


In [10]:
class GenerateExampleWith2Assert(dspy.Module):
  def __init__(self):
    super().__init__()
    self.generate_example = dspy.ChainOfThought(TranscriptExample)
  
  def forward(self):
    ex = self.generate_example(**varying_temp())
    dspy.Assert(check_format(ex.action_items), "Action Items should be a comma-separated list")
    dspy.Assert(are_action_items_included(ex.transcript, ex.action_items), "Action Items should be included in the transcript")

    return ex
  
generate_with_assert = assert_transform_module(GenerateExampleWith2Assert(), backtrack_handler)

example = generate_with_assert()
print(example)

ValueError: ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'comply': "ValueError('Respond with true or false')"})

In [ ]:
# Seems like dspy is trying to fix this for non-OpenAI models.